In [1]:
from os import statvfs_result
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2022-03-16 22:26:45.194264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-16 22:26:45.194292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

seed = 43
gamma = 0.99
max_steps_per_episode = 20000

env = gym.make('MountainCar-v0')
env.seed(seed)

[43]

In [3]:
# Neural Network
num_inputs = 2
num_actions = 3
num_hidden = 128

In [4]:
inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
common = layers.Dense(num_hidden, activation="relu")(common)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])

2022-03-16 22:27:20.755107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-16 22:27:20.755140: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-16 22:27:20.755163: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (PC): /proc/driver/nvidia/version does not exist
2022-03-16 22:27:20.755597: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
hubber_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

In [7]:
while True:
    state = env.reset()
    episode_reward = 0
    
    with tf.GradientTape() as tape:
        while True:
            if episode_count % 100 == 0:
                env.render()

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0,0])

            print(action_probs)
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            state, reward, done, _ = env.step(1)
            
            # print(action)
            # print(state)

            if reward != -1:
                reward = 1000
            else:
                reward = 0

            # if state[1] > -0.4:
            #     reward += (10*abs(state[1])) 
            reward += abs(state[1])

            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        running_reward = 0.05 * episode_reward + (1-0.05) * running_reward

        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns))
        returns = returns.tolist()

        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []

        for log_prob, value, ret in history:
            diff = ret - value
            actor_losses.append(log_prob * diff)

            critic_losses.append(
                hubber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        loss_value = sum(actor_losses) + sum(critic_losses)
        # print(loss_value)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at espisode {}"
        print(template.format(running_reward, episode_count))

    # if running_reward > 150:
    #     print("Solved at episode {}!".format(episode_count))
    #     break

tf.Tensor([[0.15422636 0.75060785 0.09516579]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.1542006  0.7506544  0.09514502]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15412459 0.75079584 0.09507959]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.153999   0.7510308  0.09497011]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15382488 0.75135756 0.09481752]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15360354 0.75177336 0.09462307]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15333678 0.75227475 0.09438845]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15302671 0.7528576  0.09411564]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15267576 0.7535173  0.09380689]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15228675 0.75424844 0.09346482]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15186273 0.75504506 0.09309228]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15140702 0.7559007  0.09269229]], shape=(1, 3), dtype=float32)
tf.Tensor([[0.15092318 0.75680864 0.09226815]], shape=(1, 3), dtype=float32)

KeyboardInterrupt: 